# Linear Programming (Linear Optimisation)

## Google Colab Settings

In [3]:
try:
  import google.colab
  COLAB = True
except:
  COLAB = False


if COLAB:
    ! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
    ! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
    ! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
    import sys
    sys.path.append('/usr/local/lib/python3.7/site-packages/')

    ! conda install -c conda-forge pyomo -y
    ! conda install -c conda-forge pyomo.extras -y
    ! conda install -c conda-forge coincbc -y
    ! conda install -c conda-forge ipopt -y
    ! conda install -c conda-forge glpk -y

## Imports

In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

## Basic LP Problem: Example 1

### Problem statement
- **We are holding a corporate pizza & prosecco party in a division and we need to optimise how many pizza and prosecco to buy**
    - We have 20 people
    - Each pizza costs \$30
    - Each prosecco costs \$50
    - Every staff drinks at least half a bottle of prosecco
    - Every staff eats at least 2 slices of pizza

### Step 1: Define Decision Variables

In [2]:
# Instantiate model and assign to model object
model = ConcreteModel()

In [3]:
# Define variables
model.n_pizza = Var(domain=NonNegativeReals)
model.n_prosecco = Var(domain=NonNegativeReals)

### Step 2: Define Objective Function

In [4]:
# Number of people 
n_people = 20

In [5]:
# Unit costs
unit_costs = {
    'pizza': 30,
    'prosecco': 50
}

In [6]:
# Objective function 
model.cost = Objective(expr=unit_costs['pizza']*model.n_pizza + \
                       unit_costs['prosecco']*model.n_prosecco)

### Step 3: Define Constraints

In [7]:
# Contraint 1: each person drinks at least half a bottle of prosecco 
model.prosecco_demand = Constraint(expr = model.n_prosecco >= 0.5 * n_people)

# Constraint 2: each person eats at least 2 slices of pizza
# Each has 6 slices
model.pizza_demand = Constraint(expr = model.n_pizza >= (2/6) * n_people)

### Step 4: Solve 

In [8]:
# Solve linear model
# Currenly using GPLK
# Other powerful commercial optimisers can be used like Gurobi or CPLEX
results = SolverFactory('glpk').solve(model)

results.write()
if results.solver.status == 'ok':
    model.pprint()


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 700.0
  Upper bound: 700.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 3
  Number of nonzeros: 3
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.022398948669433594
# ----------------------------------------------------------
#   Solution Information
# ------------------------------------

### Step 5: Analyse

In [9]:
# Display solution
print('\nCost = ', model.cost())

print('\nDecision Variables')
print('Number of Pizza = ', model.n_pizza())
print('Number of Prosecco = ', model.n_prosecco())

print('\nConstraints')
print('Pizza Demand  = ', model.pizza_demand())
print('Prosecco Demand = ', model.prosecco_demand())


Cost =  700.0000000000001

Decision Variables
Number of Pizza =  6.66666666666667
Number of Prosecco =  10.0

Constraints
Pizza Demand  =  6.66666666666667
Prosecco Demand =  10.0


## Basic LP Problem: Example 2

### Problem Statement
- **We have quant developers in 10 countries and we want to have 24 hours trading coverage**
    - Original QD count
        - Singapore: 4
        - London: 4
        - New York: 4
        - Vietnam: 3
        - India: 3
        - Hong Kong: 2
        - Indonesia: 2
        - Malaysia: 2
        - Taiwan: 2
        - Korea: 3
    - Asia coverage: we must have at least 6 quant developers at any point in time
        - Only Singapore, Vietnam, India, Hong Kong, Indonesia, Malaysia, Taiwan and Korea can handle Asia time zone
    - UK coverage: we must have at least 4 quant developers at any point in time
        - Only London and New York can handle UK time zone
        - Singapore, Hong Kong, India and Vietnam can handle UK time zone at 20% extra compensation
    - US coverage: we must have at least 4 quant developers at any point in time
        - Only London and New York can handle US time zone
        - Singapore, Hong Kong and India can handle US time zone at 30% extra compensation
    - New QD count: after leave/medical/resignations
        - Singapore: 1
        - London: 1
        - New York: 1
        - Vietnam: 3
        - India: 3
        - Hong Kong: 2
        - Indonesia: 2
        - Malaysia: 2
        - Taiwan: 2
        - Korea: 3

In [10]:
# Problem frame
# Different number of quant developers in each LBU

# Decision variables
# How many quant devs per country

# Constraints
# Cover 24 hours trading day covering SG/HK, JP, UK, US

# At least 2 quant dev covering each major zone